In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors, cm

In [2]:
# from ipywidgets import HTML
# from ipyleaflet import Polygon, Circle, Marker, MarkerCluster, AwesomeIcon, Popup
# from ipyleaflet import Map, MeasureControl, basemaps, ImageOverlay
# from ipyleaflet import MeasureControl, ScaleControl, SearchControl
# from ipywidgets import Layout

In [3]:
import folium
from folium import Map
from folium.plugins import MeasureControl, MarkerCluster

In [4]:
import pickle
from pathlib2 import Path
def read_pickle(fname):
    return  pickle.loads(Path(fname).read_bytes())

def write_pickle(data, fname):
    with open(fname, 'wb') as fid:
        pickle.dump(data, fid, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
domain = np.genfromtxt('../cvm/borders_large.txt')
domain = np.flip(domain, axis=1)
domain = [tuple(d) for d in domain]

sites = pd.read_csv("results/df_sites.csv")
print(sites.head())

          lon        lat  idx_x  idx_y   vs30          vs   elevation  \
0 -117.931099  33.908600   9833   4972  324.0  259.230011   97.820633   
1 -117.955704  33.931499   9412   5037  353.0  235.769989   86.594948   
2 -117.923721  33.945389   9575   5420  328.0  213.000000  159.873169   
3 -117.896301  33.933102   9928   5490  423.0  235.769989  117.233238   
4 -117.922508  33.871700  10234   4640  309.0  235.769989   52.210217   

       rhyp    gtl  west  
0  5.782308  False  east  
1  6.226369  False  west  
2  6.456924  False  east  
3  6.780003  False  east  
4  7.763477  False  east  


In [33]:
df_sites.loc['NP707']

site_name          NP707
lon          -117.448997
lat               33.854
idx_x              14614
idx_y               7934
vs30               533.0
vs           2913.905518
elevation     407.875702
rhyp           45.568334
gtl                False
west                east
Name: NP707, dtype: object

In [29]:
idx = pd.IndexSlice
%store -r df_sites
df_sites = df_sites.reset_index().set_index('site_name', drop=False, inplace=False)

df = read_pickle("results/df.pickle")
site_info = df_sites.columns.tolist()
df_sites_topo = pd.read_csv("../la_habra_large_gpu_abc50/stat_name_idx.txt",
                            delimiter=" ", names=["site_name", "idx_x", "idx_y"])
df_sites_topo['site_name'] = df_sites_topo['site_name'].str.replace('_', '')
df_sites_topo.set_index('site_name', drop=0, inplace=True)
def get_var_metric(models, fs, metric, ref_model='rec'):
    models = [model for model in models if model != ref_model]
    data = df.loc[:, idx[metric, fs, models + [ref_model] + site_info]] \
        .droplevel([0], axis=1)
    for f in fs:
        for model in models:
            data[f, model] = data[f, model] / data[f, ref_model]
        out = data.unstack().reset_index([0,1])  # Move freq and model to columns
        out = out.join(df_sites if 'site_name' not in df_sites.columns \
                       else df_sites.set_index('site_name', drop=False))
        out.rename(columns={out.columns[2]: metric}, inplace=True)
    return out

In [30]:
metric = 'pgv'
fs = [(0.15, 1), (0.15, 2.5), (2.5, 5), (0.15, 5)]  # avoid Pandas split the tuple
fs = [(2.5, 5)]
models = [
#     'rec',
#     'topo_q100f08_orig_vs500',
#     'topo_q50f08_s05h005l100_vs500',
#     "topo_q50f08_orig_vs500",
#     "topo_q50f08_vs30_vs500",
    'topo_q50f08_vs30erf100_vs500',
#     'topo_q50f08_vs30erf200_vs500',
#     'topo_q100f06_vs30_vs500',
#     'topo_q100f06_orig_cap_vs500',
#     'q100f06_orig_vs500',
]
data = get_var_metric(models, fs, metric, ref_model='rec')
data = data[data['model'] != 'rec']

norm = colors.Normalize(vmin = data.pgv.min(), vmax=data.pgv.max())
color = ['lightgreen', 'green', 'lightblue', 'blue', 'lightgray', 'white','beige', 'lightred', 'red', 'black']
cmap = lambda x: color[int(np.log(x / data.pgv.min() - 1e-6) * len(color) \
                           / np.log(data.pgv.max() / data.pgv.min()))]
data.head(2)

,freq,model,pgv,site_name,lon,lat,idx_x,idx_y,vs30,vs,elevation,rhyp,gtl,west
CE12919,"(2.5, 5)",topo_q50f08_vs30erf100_vs500,0.354651,CE12919,-116.972603,33.929901,18172,12266,333.0,1559.192139,795.500122,88.764685,False,east
CE13066,"(2.5, 5)",topo_q50f08_vs30erf100_vs500,0.575095,CE13066,-117.956802,33.840099,10205,4049,288.0,160.479996,35.906235,10.650173,False,west


In [31]:
clon = -117.9
clat = 34
src_lon = -117.932587 
src_lat = 33.918633
#m = Map(basemap=basemaps.Stamen.Terrain, center=(clat, clon), zoom=8.5)
m = folium.Map(location=[clat, clon], tiles="Stamen Terrain", zoom_start=8.5)
m.add_child(MeasureControl())

folium.Marker([src_lat, src_lon], pop_up='La Habra',
             icon=folium.Icon(icon='star')).add_to(m)

# for idx, site in sites.iterrows():
#     folium.Marker(location=[site['lat'], site['lon']],
#                 popup=folium.Popup(f'{site["site_name"]}: Rhypo={site["rhyp"]:.2f} km,'
#                      f'Vs30={site["vs30"]:.2f} m/s, Elev={site["elevation"]:.2f}', parse_html=True, max_width=100),
#                draggable=False).add_to(m)
    
from folium.plugins import MarkerCluster
marker_cluster = folium.plugins.MarkerCluster(
    name='259 clustered sites',
    overlay=False,
    control=False,
    icon_create_function=None
)

for idx, site in data.iterrows():  
    location = site['lat'], site['lon']
    marker = folium.Marker(location=location,
                          icon=folium.Icon(color=cmap(site.pgv)))
    popup = f'{site["site_name"]}: Rhypo={site["rhyp"]:.2f} km,' + \
            f'Vs30={site["vs30"]:.2f} m/s, Vs={site.vs:.2f} m/s, ' + \
            f'ratio={site.pgv:.2f}, Elev={site["elevation"]:.2f}'
    folium.Popup(popup).add_to(marker)
    marker_cluster.add_child(marker)
   
marker_cluster.add_to(m)
folium.LayerControl().add_to(m);


m

In [24]:
clon = -117.9
clat = 34
src_lon = -117.932587 
src_lat = 33.918633
m = Map(basemap=basemaps.Stamen.Terrain, center=(clat, clon), zoom=8.5,layout=Layout(width='960px', height='720px'))

polygon = Polygon(
    locations = domain,
    color = 'blue',
    width = 0.1,
    fill_color = 'blue',
    fill_opacity = 0.05
)

epic = Marker(
    location = (src_lat, src_lon),
    draggable = False,
    icon = AwesomeIcon(name='star', marker_color='red', icon_color='red')
)
popup = Popup(
    location = (src_lat, src_lon),
    child = HTML(value=f'({src_lon:.2f}, {src_lat:.2f})'),
    close_buttion = False,
    auto_close = False,
    close_on_escape = False,
    max_width=100, min_width=2
)


statmarkers=[]
for idx, site in sites.iterrows():
    c = Marker(location=(site['lat'], site['lon']),
               title=f'{site["site_name"]}: Rhypo={site["rhyp"]:.2f} km,'
                     f'Vs30={site["vs30"]:.2f} m/s, Elev={site["elevation"]:.2f}',
               draggable=False)
    statmarkers.append(c)

# image = ImageOverlay(
#     url = "../cvm/test.png",
#     bounds = ((33.3, -119.2),(33.8, -118.5))
# )
# m.add_layer(image)


m.add_layer(popup)
m.add_layer(epic)
m.add_layer(polygon)
m.add_layer(MarkerCluster(markers=statmarkers))
m

Map(center=[34, -117.9], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [7]:
import pickle
%matplotlib widget
with open('results/syn_sites.pickle', 'rb') as fid:
    syn_sites = pickle.load(fid)
    
nx, ny, nz = 19440, 14904, 184
dh = 0.008
srcidx = [9681, 5086, 283]

    
z200 = np.fromfile('z200.bin', dtype='float32').reshape(ny, nx)
z500 = np.fromfile('z500.bin', dtype='float32').reshape(ny, nx)
fig, ax = plt.subplots(dpi=100)
stepx, stepy = 20, 20
rx = np.arange(0, nx, stepx) * dh 
ry = np.arange(0, ny, stepy) * dh 

#im = ax.contourf(rx, ry, z200[::stepy, ::stepx], cmap='YlOrBr')
im = ax.contourf(rx, ry, z200[::stepy, ::stepx], 3, cmap='YlOrBr')
# ax.contour(rx, ry, z500[::stepy, ::stepx], [0, 10], cmap='gray_r',
#            linestyles='dashed', linewidths=0.8)
ax.contour(rx, ry, z500[::stepy, ::stepx], 3, cmap='twilight_shifted',
           linewidths=0.6)
coastline = np.genfromtxt('coastline.idx', dtype='int', usecols=[1,2])
coastline = coastline[coastline[:, -1] >= 0, :]
ax.plot(coastline[:, 0] * dh, coastline[:, 1] * dh, 'k--')
cbar=plt.colorbar(im)
cbar.set_ticks([0, 8, 16, 24])
ax.scatter(srcidx[0] * dh, srcidx[1] * dh, 200, alpha=0.8, color='k', marker='*')
for i, s in enumerate(syn_sites):   
    ax.scatter(s[1] * dh, s[2] * dh, 15, color='cyan', marker='^')
    ax.annotate(s[0], (s[1] * dh, s[2] * dh), xytext=(4, 0), textcoords='offset points',
                   color='k', ha='left', va='center', fontsize='x-small', fontweight='light')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
sites.describe()

,lon,lat,idx_x,idx_y,vs30,elevation,rhyp
count,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000
mean,-117.906172,33.967717,9530.795367,5793.849421,365.169884,236.166750,34.434289
std,0.362966,0.173317,3629.258464,3187.801987,90.052838,344.690541,18.370473
min,-118.649712,33.370998,1100.000000,829.000000,220.000000,2.363467,1.124054
25%,-118.195850,33.853750,6590.000000,3318.000000,305.500000,40.744595,22.411951
50%,-117.959099,33.976200,9147.000000,4937.000000,353.000000,108.007446,31.686805
75%,-117.658264,34.078701,12121.000000,7765.000000,406.500000,288.750214,41.651250
max,-116.953003,34.540001,18419.000000,13997.000000,881.000000,2258.416260,90.483835


In [8]:
clon = -117.9
clat = 34
src_lon = -117.932587 
src_lat = 33.918633
m = Map(basemap=basemaps.Stamen.Terrain, center=(clat, clon), zoom=8.5,layout=Layout(width='960px', height='720px'))

polygon = Polygon(
    locations = domain,
    color = 'blue',
    width = 0.1,
    fill_color = 'blue',
    fill_opacity = 0.05
)

epic = Circle(
    location = (src_lat, src_lon),
    draggable = False,

    #icon = AwesomeIcon(name='star', marker_color='red', icon_color='red')
)

popup = Popup(
    location = (src_lat, src_lon),
    child = HTML(value=f'({src_lon:.2f}, {src_lat:.2f})'),
    close_buttion = False,
    auto_close = False,
    close_on_escape = False,
    max_width=100, min_width=2
)

# statmarkers=[]
# for idx, site in sites[0:50:10].iterrows():
#     c = Marker(location=(site['lat'], site['lon']),
#                title=f'Rhypo={site["rhypo"]:.2f} km, Vs30={site["vs30"]:.2f} m/s',
#                draggable=False)
#     statmarkers.append(c)
# measure = MeasureControl(
#     location=
# )
# m.add_layer(MarkerCluster(markers=statmarkers))

image = ImageOverlay(
    url = "../cvm/test.png",
    bounds = ((33.3, -119.2),(33.8, -118.5))
)
m.add_layer(image)
m.add_layer(epic)
m.add_layer(popup)
m.add_layer(polygon)
m.add_control(ScaleControl(position='bottomleft'))
display(m)
# measure = MeasureControl(
#     position='bottomleft',
#     active_color = 'orange',
#     primary_length_unit = 'kilometers'
# )


Map(center=[34, -117.9], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [36]:
eql=pd.read_csv("aftershock_list_m4+.txt", delimiter="_", header=None, parse_dates=[[0]])
eql.columns=["Date", "Mw", "Lat", "Lon", "Depth"]
#eql.set_index("id", drop=True, inplace=True)
eql["Mw"]=eql["Mw"].str.rstrip("Mw").astype(np.float)
eql["Lat"]=eql["Lat"].str.rstrip("S").astype(np.float).map(lambda x: -x)
eql["Lon"]=eql["Lon"].str.rstrip("W").astype(np.float).map(lambda x: -x)
eql["Depth"]=eql["Depth"].str.rstrip("KM").astype(np.float)

In [37]:
def makebasemap(clon=-70, clat=-35, inizoom=6, width="960px", height="720px"):
    defaultLayout=Layout(width=width, height=height)
    bmap = Map(basemap=basemaps.Stamen.Terrain, center=(clat, clon), zoom=inizoom, layout=defaultLayout)
    measure = MeasureControl(position='bottomleft', active_color = 'orange',
        primary_length_unit = 'kilometers')    
    bmap.add_control(measure)
    bmap.add_layer(polygon)
    return bmap

In [38]:
def plot_earthquakes(eqdf, bmap, htype="circle", circle_color="red", circrad=1.e4,
                     lonkey="Longitude", latkey="Latitude"):
    
    
    if htype == "marker" or htype == "cluster":
        if htype == "cluster":
            statmarkers=[]

        for eqid in eqdf.index:
            c = Marker(location=(eqdf.loc[eqid,latkey], eqdf.loc[eqid,lonkey]),
                title="Mw=%3.1f (%s, %3.0f km)" % (eql.loc[eqid,"Mw"], eql.loc[eqid,"Date"], eql.loc[eqid,"Depth"]),
                draggable=False)
            
            if htype == "cluster":
                statmarkers.append(c)
            else:
                bmap.add_layer(c)

        if htype == "cluster":
            mc=MarkerCluster(markers=statmarkers)
            bmap.add_layer(mc)
        
    elif htype == "circle":
        for eqid in eqdf.index:
            c = Circle()
            c.location = (eqdf.loc[eqid,latkey], eqdf.loc[eqid,lonkey])
            c.radius = int((eqdf.loc[eqid,"Mw"] - 4)*circrad)
            c.color = circle_color
            c.fill_color = circle_color
            c.fill_opacity = 0.1
            
            bmap.add_layer(c)

In [39]:
bmap2=makebasemap()
#mask=eql["Date"] > "2011-01-01"
mask=(eql["Date"] > "2011-01-01") & (eql["Mw"] > 6.8)
plot_earthquakes(eql[mask], bmap2, htype="cluster", latkey="Lat", lonkey="Lon")
display(bmap2)

Map(center=[-35, -70], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [67]:
eql.loc[eqid,:]

,Mw,Lat,Lon,Depth
id,,,,
20100227,6.2,-34.32,71.22,46.0
20100227,8.8,-36.10,73.08,30.0


In [99]:
!ls *.pkl

20100227_6.2Mw_34.32S_71.22W_46KM.pkl 20170424_6.9Mw_33.09S_72.09W_24KM.pkl
20100227_8.8Mw_36.10S_73.08W_30KM.pkl 20180615_4.1Mw_20.04S_69.05W_89KM.pkl
20120325_6.8Mw_35.2S_72.22W_40KM.pkl  20190801_6.6Mw_34.28S_72.51W_13KM.pkl
20140823_6.4Mw_32.74S_71.5W_40KM.pkl  domain_edges_v1.pkl


In [106]:
Marker?

In [41]:
def stat_df_from_pickle(picklefile):
    ds=pickle.load(open(picklefile, "rb"))
    
    lons=[]
    lats=[]
    names=[]

    for st in ds.keys():
        lats.append(ds[st]["lat"])
        lons.append(ds[st]["lon"])
        names.append(ds[st]["name"])
        
    statdf=pd.DataFrame({"name": names, "Longitude": lons, "Latitude": lats})
    statdf.set_index("name", inplace=True)
    
    return statdf

In [42]:
def plot_event_stations(statdf, bmap, cluster=True, epi=(0,0), epiradius=10000, popup=False):

    if cluster:
        statmarkers=[]

    for name in statdf.index:
        c = Marker(location=(statdf.loc[name,"Latitude"], statdf.loc[name,"Longitude"]),
            title=name, draggable=False)
        c.name=name
        c.popup=HTML(value=name)
        
        if cluster:
            statmarkers.append(c)
        else:
            bmap.add_layer(c)
            
        if popup:
            message1 = HTML()
            message1.value = name

            # Popup with a given location on the map:
            popup = Popup(location=(statdf.loc[name,"Latitude"]+0.01, statdf.loc[name,"Longitude"]),
                        child=message1, close_button=False, auto_close=False, close_on_escape_key=False,
                         max_width=10, min_width=1)
            bmap.add_layer(popup)

    if cluster:
        mc=MarkerCluster(markers=statmarkers)
        bmap.add_layer(mc)
        
    print(epi)
    cir=Circle(location=epi, radius=epiradius, color="red", fill_color="red", fill_opacity=0.1)
    bmap.add_layer(cir)

In [43]:
maulesites=pd.read_pickle("maulesites_corr.pkl")
maulesites.head()

FileNotFoundError: [Errno 2] No such file or directory: 'maulesites_corr.pkl'

In [257]:
!ls -l 20*.pkl

-rw-rw-r--@ 1 daniel  staff    404603 Aug  6 17:59 20100227_6.2Mw_34.32S_71.22W_46KM.pkl
-rw-r--r--@ 1 daniel  staff   9851094 Nov  2 21:57 20100227_8.8Mw_36.10S_73.08W_30KM.pkl
-rw-rw-r--@ 1 daniel  staff   3605178 Aug  6 17:59 20120325_6.8Mw_35.2S_72.22W_40KM.pkl
-rw-rw-r--@ 1 daniel  staff  30817914 Aug  6 17:59 20140823_6.4Mw_32.74S_71.5W_40KM.pkl
-rw-rw-r--@ 1 daniel  staff  33694253 Aug  6 17:59 20170424_6.9Mw_33.09S_72.09W_24KM.pkl
-rw-r--r--@ 1 daniel  staff  19726692 Dec  8 22:45 20170802_5.5Mw_33.2S_70.61W_99KM.pkl
-rw-r--r--@ 1 daniel  staff   2322256 Nov  2 21:45 20180615_4.1Mw_20.04S_69.05W_89KM.pkl
-rw-rw-r--@ 1 daniel  staff   7178592 Aug  6 17:59 20190801_6.6Mw_34.28S_72.51W_13KM.pkl


In [40]:
bmap=makebasemap(clat=-33.4, clon=-70.5, inizoom=10, height="480px")
#pfile="20170802_5.5Mw_33.2S_70.61W_99KM.pkl"
#pfile="20100227_8.8Mw_36.10S_73.08W_30KM.pkl"
pfile="20170424_6.9Mw_33.09S_72.09W_24KM.pkl" #offshore valparaiso
statdf=stat_df_from_pickle(pfile)
epiloc=-float(pfile.split("_")[2].split("S")[0]), -float(pfile.split("_")[3].split("W")[0])
plot_event_stations(statdf, bmap,epi=epiloc, epiradius=1000, cluster=False)
display(bmap)

NameError: name 'stat_df_from_pickle' is not defined

In [321]:
#ma=Marker()
ma.name

''

In [222]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians
"""
bsas = [-34.83333, -58.5166646]
paris = [49.0083899664, 2.53844117956]
bsas_in_radians = [radians(_) for _ in bsas]
paris_in_radians = [radians(_) for _ in paris]
print(bsas_in_radians)
print(paris_in_radians)
result = haversine_distances([bsas_in_radians, paris_in_radians])
result * 6371000/1000  # multiply by Earth radius to get kilometers
""";

In [348]:
def find_matching(df1, df2, threshold=1000.):
    earth_radius=6371000
    nstat1=len(df1)
    nstat2=len(df2)
    
    mpairs=[]
    
    for n in df1.index:
        for m in df2.index:
            stat1=[radians(df1.loc[n,"Latitude"]),radians(df1.loc[n,"Longitude"])]
            stat2=[radians(df2.loc[m,"Latitude"]),radians(df2.loc[m,"Longitude"])]
            distances=haversine_distances([stat1,stat2]) * earth_radius 
            if distances[0][1] < threshold:
                mpairs.append([n, m, distances[0][1]])
                
    return(mpairs)
            
find_matching(statdf, maulesites, threshold=3000.)

[['R18M', 'CRMA', 2152.012050782631],
 ['R05M', 'LACH', 867.4850197528216],
 ['R02M', 'F', 962.3141308262449],
 ['MT07', 'ROC1', 37.31250874947311],
 ['V01A', 'VAL', 1770.8995364335185]]

In [381]:
#maulesites.loc['LACH',:]
statdf.loc['R02M',:]

Longitude   -70.6597
Latitude    -33.4728
Name: R02M, dtype: float64

In [345]:
bmap2=makebasemap(clat=-33.4, clon=-70.5, inizoom=10, height="480px")
plot_event_stations(maulesites, bmap2, cluster=False)
display(bmap2)

(0, 0)


Map(center=[-33.4, -70.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [179]:
len(statdf)
#len(maulesites.join(statdf, lsuffix="l", rsuffix="r"))

14

In [109]:
c?

In [115]:
ds=pickle.load(open("20100227_8.8Mw_36.10S_73.08W_30KM.pkl", "rb"))
maulestations=set([ds[st]["name"] for st in ds.keys()])

ds=pickle.load(open("20190801_6.6Mw_34.28S_72.51W_13KM.pkl", "rb"))
stat2019=set([ds[st]["name"] for st in ds.keys()])

In [116]:
maulestations

{'ANGOL',
 'CONCEPCION CENTRO',
 'CONSTITUCION',
 'CURICO',
 'HUALANE',
 'LLOLLEO',
 'MATANZAS',
 'SANTIAGO CENTRO',
 'SANTIAGO LA FLORIDA',
 'SANTIAGO MAIPU',
 'SANTIAGO PENALOLEN',
 'SANTIAGO PUENTE ALTO',
 'TALCA',
 'VALPARAISO ALMENDRAL',
 'VINA DEL MAR CENTRO',
 'VINA DEL MAR EL SALTO'}

In [117]:
stat2019

{'BO01',
 'BO02',
 'BO03',
 'LMEL',
 'ML02',
 'MT01',
 'MT05',
 'MT09',
 'MT10',
 'MT14',
 'MT15',
 'MT18',
 'VA03',
 'VA05'}

In [378]:
eql[(eql["Date"] > "2010/01/01") & (eql["Date"] < "2010/12/31")]

,Date,Mw,Lat,Lon,Depth
69,2010-02-27,6.2,-34.32,-71.22,46.0
70,2010-02-27,8.8,-36.10,-73.08,30.0
71,2010-02-28,6.2,-34.90,-71.62,46.0


In [396]:
statdf.to_pickle("../sdc_55/stations.pkl")
pickle.dump(epiloc, open("../sdc_55/hypo.pkl", "wb"))

In [405]:
#statdf.to_pickle("../valparaiso_69/stations.pkl")
#pickle.dump(epiloc, open("../valparaiso_69/hypo.pkl", "wb"))